In [ ]:
import numpy as np

In [ ]:
with open("input.txt") as f:
    data = [line.strip() for line in f.readlines()]

In [ ]:
seeds = [int(seed) for seed in data[0].split(": ")[1].split()]
seeds

In [ ]:
il = iter(data[1:])
maps = dict()
mapnames = dict()
index = 0
for line in il:
    if not line.strip():
        continue
    if 'map' in line:
        mapname = line.strip().split()[0]
        mapnames[index] = mapname
        M = []
        while True:
            try:
                line = next(il)
            except StopIteration:
                break
            if not line.strip():
                break
            M.append(list(map(int, line.split())))
        Ma = np.array(M)
        maps[index] = Ma[np.argsort(Ma[:, 1])]
        index += 1
print(mapnames)

In [ ]:
# maps work destinationid, sourceid, range

In [ ]:
rangemaps = dict()

In [ ]:
map1 = maps[0]

In [ ]:
seeds[0]

In [ ]:
map1

In [ ]:
bin_location = np.searchsorted(map1[:,1], seeds[0], side='left')
bin_location

In [ ]:
map1[bin_location-1], seeds[0]

In [ ]:
def is_between(value, lower_bound, upper_bound):
    return lower_bound <= value < upper_bound
destid, sourceid, r = map1[bin_location-1]
print(sourceid, sourceid+r, seeds[0])
is_between(seeds[0], sourceid, sourceid+r)

In [ ]:
maps.keys()

In [ ]:
def find_location(maps, seed, verbose=True):
    if verbose:print("---", seed)
    for i in range(len(maps)):
        if verbose: print(i, mapnames[i])
        bin_location = np.searchsorted(maps[i][:,1], seed, side='right')
        destid, sourceid, r = maps[i][bin_location-1]
        if is_between(seed, sourceid, sourceid+r):
            if verbose: print(f"seed {seed:d} is mapped between sourceid {sourceid} which ranges until {sourceid+r}")
            diff = seed - sourceid
            seed = destid + diff
        else:
            if verbose:
                print(f"seed {seed} is not within range {sourceid}-{sourceid+r} so it will map to its own value")
                print(bin_location)
                print(maps[i][bin_location-1:bin_location+3])
    return seed

In [ ]:
find_location(maps, seeds[0])

In [ ]:
locations = [ find_location(maps, seed, verbose=False) for seed in seeds ]

In [ ]:
min(locations)

## question 2

In [ ]:
ranges = np.c_[seeds[0::2], np.sum([seeds[0::2], (np.array(seeds[1::2])-1)], axis=0)]
ranges

In [ ]:
def check_range(maps, startid, finalid):
    for i in range(len(maps)):
        
        split_range(maps[i], startid, finalid)
        
        # get bin_location of startid
        bin_location = np.searchsorted(maps[i][:,1], startid, side='right')
        destid, sourceid, r = maps[i][bin_location-1]
        
        if is_between(seed, sourceid, sourceid+r):
            if verbose: print(f"seed {seed:d} is mapped between sourceid {sourceid} which ranges until {sourceid+r}")
            diff = seed - sourceid
            seed = destid + diff
        else:
            if verbose:
                print(f"seed {seed} is not within range {sourceid}-{sourceid+r} so it will map to its own value")
                print(bin_location)
                print(maps[i][bin_location-1:bin_location+3])
    return seed

In [ ]:
def split_range(xmap, startid, finalid):
    """ this function will look whether startid and finalid fall within the same subrange and otherwise return 
    multiple ranges that map to different destination ids"""
    bin_location = np.searchsorted(xmap[:,1], startid, side='right')
    print("bin_location", bin_location)
    destid, sourceid, r = xmap[bin_location-1]
    
    if is_between(startid, sourceid, sourceid+r):
        print(f"start value {startid} falls within sourceid {sourceid} and {sourceid+r}")
        diff = startid - sourceid
        seed = destid + diff
    else:
        try:
            destid2, sourceid2, r2 = xmap[bin_location]
        except IndexError:
            # in this case there is no higher bin, so map to itself
            return [[startid, finalid]]
            
        print(f"start value {startid} falls outside range. next range starts at {sourceid2}")
        # check if final id also falls outside range
        if finalid < sourceid2:
            print(f"whole seed range {startid}-{finalid} falls outside ranges {sourceid}-{sourceid+r} and {sourceid2}-{sourceid2+r2}")
            return [[startid, finalid]]
        else:
            # sourceid2 < finalid. new range is sourceid2-finalid
            return [[startid, sourceid2-1]] + split_range(xmap, sourceid2, finalid)
    
    if (sourceid + r) > finalid:
        
        print(f"the seed range {startid}-{finalid} falls totally within range {sourceid}-{sourceid+r}")
        return [[seed, seed + (finalid-startid)]]
    else:
        # now sourceid + r < finalid. so we have to check the new range sourceid + r + 1 - finalid
        print(f"the seed range {startid}-{finalid} is larger than range {sourceid}-{sourceid+r-1}")
        print(f"new range {sourceid+r}-{finalid}")
        return [[seed, destid+r-1]] + split_range(xmap, sourceid+r, finalid)

In [ ]:
new_ranges = ranges.copy()
for i in range(len(maps)):
    new_new_ranges = []
    for xrange in new_ranges:
        new_range = split_range(maps[i], *xrange)
        new_new_ranges.extend(new_range)
    new_ranges = new_new_ranges.copy()

In [ ]:
np.min(new_ranges)

In [ ]:
def is_between(value, lower_bound, upper_bound):
    return lower_bound <= value < upper_bound


def split_range(xmap, startid, finalid):
    """ this function will look whether startid and finalid fall within the same subrange and otherwise return 
    multiple ranges that map to different destination ids"""
    bin_location = np.searchsorted(xmap[:,1], startid, side='right')
    destid, sourceid, r = xmap[bin_location-1]
    
    if is_between(startid, sourceid, sourceid+r):
        diff = startid - sourceid
        seed = destid + diff
    else:
        try:
            destid2, sourceid2, r2 = xmap[bin_location]
        except IndexError:
            return [[startid, finalid]]
            
        if finalid < sourceid2:
            return [[startid, finalid]]
        else:
            return [[startid, sourceid2-1]] + split_range(xmap, sourceid2, finalid)
    
    if (sourceid + r) > finalid:
        return [[seed, seed + (finalid-startid)]]
    else:
        return [[seed, destid+r-1]] + split_range(xmap, sourceid+r, finalid)
    
new_ranges = ranges.copy()
for i in range(len(maps)):
    new = []
    for xrange in new_ranges:
        new_range = split_range(maps[i], *xrange)
        new.extend(new_range)
    new_ranges = new.copy()

print(min(new_ranges))